<a href="https://colab.research.google.com/github/labintrieri/estudo/blob/main/Exerc%C3%ADcio_Minera%C3%A7%C3%A3o_de_Dados_Rob%C3%B4_Raspador_BBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importando bibliotecas
import requests
import bs4
import time


In [ ]:
#função que cria arquivos
def grava_arquivo(nome, titulo, texto):
  conteudo = titulo + "\n"

  for paragrafo in texto:
    conteudo = conteudo + paragrafo + "\n"

  with open(nome,'w') as f:
          f.write(conteudo)



In [ ]:
#vamos capturar o assunto "Ciência"

result = requests.get("https://www.bbc.com/portuguese/topics/cr50y580rjxt")
soup = bs4.BeautifulSoup(result.text)

links = soup.find_all('a', { 'class': 'focusIndicatorDisplayBlock bbc-uk8dsi e1d658bg0' }) #pegando links



In [ ]:
numero_de_artigos = 0 #criando variável 0 para o laço ir somando +1 depois

lista_dos_nomes_arquivos_criados = [] #criando lista vazia para povoar depois

for link in links:
    time.sleep(2)
    print(link['href'])
    result2 = requests.get(link['href'])
    soup2 = bs4.BeautifulSoup(result2.text)
    if not( (soup2.find('h1',attrs={"class":"bbc-14gqcmb e1p3vdyi0"})) is None ): #não é vídeo
          titulo = soup2.find('h1',attrs={"class":"bbc-14gqcmb e1p3vdyi0"}).text #pegar título
          texto = [] #criando lista vazia para povoar depois
          for linha in soup2.find_all('div',attrs={"class":"bbc-19j92fr ebmt73l0"}): #pegar parágrafo
            texto.append(linha.text) #povoando a lista texto
          numero_de_artigos = numero_de_artigos + 1 #laço para ir somando o número de artigos
          nome_do_arquivo = 'arq'+str(numero_de_artigos)+".txt" #usa o número de artigos para criar os arquivos
          lista_dos_nomes_arquivos_criados.append(nome_do_arquivo) #povoando a lsita dos nomes dos arquivos criados
          grava_arquivo(nome_do_arquivo, titulo, texto) #resgata a bota pra rodar a função que cria arquivos
          print("................")

print(lista_dos_nomes_arquivos_criados)

https://www.bbc.com/portuguese/articles/clmp08dj43vo
................
https://www.bbc.com/portuguese/articles/c032drl6376o
................
https://www.bbc.com/portuguese/articles/c292zegeg2yo
................
https://www.bbc.com/portuguese/articles/cn0p3xlp2p9o
................
https://www.bbc.com/portuguese/articles/c3g273zjkdpo
................
https://www.bbc.com/portuguese/articles/cprp5x929p8o
................
https://www.bbc.com/portuguese/articles/cekpy981vy0o
................
https://www.bbc.com/portuguese/articles/cyx2ddnp5l0o
https://www.bbc.com/portuguese/articles/c99enlvxv0do
................
https://www.bbc.com/portuguese/articles/c51r229y3vwo
................
https://www.bbc.com/portuguese/articles/cw02k55wp1ko
https://www.bbc.com/portuguese/articles/cxx1rwj3zzjo
................
https://www.bbc.com/portuguese/articles/cw02xy364gwo
................
https://www.bbc.com/portuguese/articles/cv2z7p43d1po
................
https://www.bbc.com/portuguese/articles/ckkp94nrkygo
.

In [ ]:
#vamos carregar a biblioteca Spacy
import spacy.cli
spacy.cli.download("pt_core_news_sm")

import spacy
nlp = spacy.load("pt_core_news_sm")

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
#vamos ler os arquivos que foram criados e começar a classificação
def le_str_arq(nome):
  arquivo = open(nome, "r")
  dados = arquivo.read()
  arquivo.close()
  return dados

for nome_arquivo in lista_dos_nomes_arquivos_criados:
  texto = le_str_arq(nome_arquivo)
  print(nome_arquivo)
  doc = nlp(texto)
  for entity in doc.ents:
      print(entity, entity.label_)


In [ ]:
import pandas as pd

In [ ]:
#criando o df que irá receber as informações
df = pd.DataFrame(columns=['artigo','entidade_nomeada','categoria'])

In [ ]:
#povoando o df
for nome_arquivo in lista_dos_nomes_arquivos_criados:
    texto = le_str_arq(nome_arquivo)
    doc = nlp(texto)
    for entity in doc.ents:
        df.loc[len(df.index)] = [nome_arquivo, entity.text, entity.label_]



In [ ]:
#vamos ver se deu certo o povoamento do df
df.shape
df.head



In [ ]:
#averiguando com mais detalhe se deu certo
print(df['entidade_nomeada'].value_counts())

In [ ]:
#vamos criar o csv
df.to_csv('ner.csv')